In [1]:
from pyamaze import maze, agent, COLOR, textLabel
from timeit import timeit
from queue import PriorityQueue

In [2]:
def h(cell1, cell2):
    x1, y1 = cell1
    x2, y2 = cell2
    return (abs(x1 - x2) + abs(y1 - y2))


def aStar(m, start=None):
    if start is None:
        start = (m.rows, m.cols)
    open = PriorityQueue()
    open.put((h(start, m._goal), h(start, m._goal), start))
    aPath = {}
    g_score = {row: float("inf") for row in m.grid}
    g_score[start] = 0
    f_score = {row: float("inf") for row in m.grid}
    f_score[start] = h(start, m._goal)
    searchPath = [start]
    while not open.empty():
        currCell = open.get()[2]
        searchPath.append(currCell)
        if currCell == m._goal:
            break
        for d in 'ESNW':
            if m.maze_map[currCell][d] == True:
                if d == 'E':
                    childCell = (currCell[0], currCell[1]+1)
                elif d == 'W':
                    childCell = (currCell[0], currCell[1]-1)
                elif d == 'N':
                    childCell = (currCell[0]-1, currCell[1])
                elif d == 'S':
                    childCell = (currCell[0]+1, currCell[1])

                temp_g_score = g_score[currCell] + 1
                temp_f_score = temp_g_score + h(childCell, m._goal)

                if temp_f_score < f_score[childCell]:
                    aPath[childCell] = currCell
                    g_score[childCell] = temp_g_score
                    f_score[childCell] = temp_g_score + h(childCell, m._goal)
                    open.put((f_score[childCell], h(
                        childCell, m._goal), childCell))

    fwdPath = {}
    cell = m._goal
    while cell != start:
        fwdPath[aPath[cell]] = cell
        cell = aPath[cell]
    return searchPath, aPath, fwdPath

In [3]:
m = maze(20, 30)

# Maze with only one path to reach goal
# m.CreateMaze(loadMaze='Maze_1_singlePath.csv', theme="light")

# Maze with multiple paths to reach goal
# m.CreateMaze(loadMaze='Maze_1_mulptiplePaths.csv', theme="light")

# Maze with only one path to reach goal @(1,30)
# m.CreateMaze(1, 30, loadMaze='Maze_1_singlePath.csv', theme="light")

# Maze with multiple paths to reach goal @(1,30)
m.CreateMaze(1, 30, loadMaze='Maze_1_mulptiplePaths.csv', theme="light")

searchPath, aPath, fwdPath = aStar(m)

textLabel(m, 'A-Star Path Length', len(fwdPath)+1)
textLabel(m, 'A-Star Search Length', len(searchPath))

a = agent(m, footprints=True, color=COLOR.yellow,
          shape='square', filled=True)
b = agent(m, footprints=True, color=COLOR.red,
          shape='square', filled=False)
# c = agent(m, 1, 1, footprints=True, color=COLOR.cyan,
#           shape='square', filled=True, goal=(m.rows, m.cols))
c = agent(m, 1, 30, footprints=True, color=COLOR.cyan,
          shape='square', filled=True, goal=(m.rows, m.cols))

m.tracePath({a: searchPath}, delay=50)
m.tracePath({c: aPath}, delay=50)
m.tracePath({b: fwdPath}, delay=50)

t2 = timeit(stmt='aStar(m)', number=1000, globals=globals())
textLabel(m, 'A-Star Time', t2)

m.run()